# script to prepare the files for loop QTL analysis

genotypes are only available through protected access

QTL was generated using QTLtools

In [1]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
os.chdir("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis")
base_dir = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis"

## prepare the normalized scores for loopQTL

In [3]:
loops_analysed = pd.read_pickle(f"{base_dir}/HiC_analysis/extracting_loop_counts/aggregated_counts/aggregated_normalized_loops_CD4_CD8.pk")
metadata_hic = pd.read_csv(f"{base_dir}/metadata/cleaned_HiC_metadata.csv", index_col = 0)

In [5]:
loops_analysed["gene_id"] = loops_analysed.index
loops_analysed["length"] = loops_analysed["B_end"] - loops_analysed["A_start"] 
loops_analysed["strand"] = "+"

/tmp/ipykernel_23348/2228809681.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  loops_analysed["gene_id"] = loops_analysed.index
/tmp/ipykernel_23348/2228809681.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  loops_analysed["length"] = loops_analysed["B_end"] - loops_analysed["A_start"]
/tmp/ipykernel_23348/2228809681.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(

In [50]:
# we don't have genotypes for NRHV319 and NRHV005
samples = metadata_hic[~metadata_hic["patient"].isin(["NRHV319", "NRHV005"])]["patient"].to_list()

## CD8 loops

In [51]:
wanted_samples = metadata_hic[(metadata_hic["cell_type"] == "CD8") & metadata_hic["patient"].isin(samples)]["folder_name"].to_list()

In [52]:
counts_CD8 = loops_analysed[["chrA", "A_start", "B_end", "gene_id", "length", "strand"] + wanted_samples].copy()

In [53]:
df_mapper = metadata_hic[(metadata_hic["cell_type"] == "CD8") & metadata_hic["patient"].isin(samples)][["folder_name", "patient"]]

In [54]:
counts_CD8[wanted_samples] = np.log2(counts_CD8[wanted_samples] + 1)

In [55]:
counts_CD8 = counts_CD8.rename(columns = dict(zip(df_mapper['folder_name'], df_mapper['patient'])))
counts_CD8 = counts_CD8.rename(columns = {"chrA":"#chr", "A_start":"start", "B_end":"end","gene_id":"gene"})
counts_CD8["#chr"] = "chr" + counts_CD8["#chr"].astype(str)

In [56]:
counts_CD8.sort_values(["#chr","start"]).to_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/loop_CD8_test.bed", sep = "\t", index = False)
!bgzip -f /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/loop_CD8_test.bed
!tabix -f -p bed /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/loop_CD8_test.bed.gz

## CD4 loops

In [ ]:
# PSA5006 and NRHV151 were identified as outliers from the PCA
wanted_samples = metadata_hic[(metadata_hic["cell_type"] == "CD4") & metadata_hic["patient"].isin(samples) & (~metadata_hic["patient"].isin(["PSA5006", "NRHV151"]))]["folder_name"].to_list()

In [58]:
counts_CD4 = loops_analysed[["chrA", "A_start", "B_end", "gene_id", "length", "strand"] + wanted_samples].copy()

In [59]:
df_mapper = metadata_hic[(metadata_hic["cell_type"] == "CD4") & metadata_hic["patient"].isin(samples)][["folder_name", "patient"]]

In [60]:
counts_CD4[wanted_samples] = np.log2(counts_CD4[wanted_samples] + 1)

In [61]:
counts_CD4 = counts_CD4.rename(columns = dict(zip(df_mapper['folder_name'], df_mapper['patient'])))
counts_CD4 = counts_CD4.rename(columns = {"chrA":"#chr", "A_start":"start", "B_end":"end","gene_id":"gene"})
counts_CD4["#chr"] = "chr" + counts_CD4["#chr"].astype(str)

In [62]:
counts_CD4.sort_values(["#chr","start"]).to_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/loop_CD4_test.bed", sep = "\t", index = False)
!bgzip -f /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/loop_CD4_test.bed
!tabix -f -p bed /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/loop_CD4_test.bed.gz

## running a permutation test to identify the best number of PCA component as covariates identified the following:
for CD8: 0 genotype covariates and 0 phenotype covariates
for CD4: 0 genotype covariates and 0 phenotype covariates

In [ ]:
# see bash scripts for running of QTL tools

In [1]:
# merge the final results
!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/loop_nominal_CD4_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/loop_nominal_CD4_merged.txt
!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/loop_nominal_CD8_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/loop_nominal_CD8_merged.txt

!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/loop_permuted_CD4_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/loop_permuted_CD4_merged.txt
!cat /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/loop_permuted_CD8_0* > /mnt/iusers01/jw01/mdefscs4/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/loop_permuted_CD8_merged.txt

In [2]:
loop_permuted_CD8 = pd.read_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/loop_permuted_CD8_merged.txt", sep = " ")
len(loop_permuted_CD8[loop_permuted_CD8["adj_beta_pval"] < 0.1])

12307

## Add FDR correction to permuted p-values

In [3]:
from statsmodels.stats import multitest
loop_permuted_CD8["FDR"] = multitest.fdrcorrection(loop_permuted_CD8["adj_beta_pval"], alpha = 0.1)[1]
loop_permuted_CD8.to_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/loop_permuted_CD8_FDR.txt", sep = " ", index = False)
len(loop_permuted_CD8[loop_permuted_CD8["FDR"] < 0.1])

762

In [4]:
loop_permuted_CD4 = pd.read_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/loop_permuted_CD4_merged.txt", sep = " ")
len(loop_permuted_CD4[loop_permuted_CD4["adj_beta_pval"] < 0.1])

11595

In [5]:
loop_permuted_CD4["FDR"] = multitest.fdrcorrection(loop_permuted_CD4["adj_beta_pval"], alpha = 0.1)[1]
loop_permuted_CD4.to_csv("/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis/QTL_analysis/HiC/output_final/loop_permuted_CD4_FDR.txt", sep = " ", index = False)
len(loop_permuted_CD4[loop_permuted_CD4["FDR"] < 0.1])

579